# CNN 심화

## [예제6] DenseNet Fine Tuning

> ### Load modules

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

print("Module Loaded.")
print("NumPy Version :{}".format(np.__version__))
print("TensorFlow Version :{}".format(tf.__version__))
print("Matplotlib Version :{}".format(plt.matplotlib.__version__))

Module Loaded.
NumPy Version :1.18.5
TensorFlow Version :2.3.0
Matplotlib Version :3.2.2


> ### Import Keras Layers

In [2]:
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization

> ### Load DenseNet Model

In [3]:
IMG_SIZE = 64

In [4]:
from tensorflow.keras.applications import DenseNet121

model = DenseNet121(
    weights="imagenet", 
    include_top=False,  # 최상위의 Dense layer 포함 여부 설정
    input_tensor=Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    )

model.summary()

29089792/29084464 [==============================] - 0s 0us/step
Model: "densenet121"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 70, 70, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 32, 32, 64)   9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 32, 32, 64)   256         conv1/conv[0][0]                 
_______________________

> ### Layer를 추가하여 모델 완성

In [5]:
from tensorflow.keras.layers import GlobalAveragePooling2D

model_fine = tf.keras.models.Sequential()
 
model_fine.add(model)
 
model_fine.add(GlobalAveragePooling2D())
model_fine.add(Dense(10, activation='softmax'))

model_fine.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Functional)     (None, 2, 2, 1024)        7037504   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                10250     
Total params: 7,047,754
Trainable params: 6,964,106
Non-trainable params: 83,648
_________________________________________________________________


> ### Load CIFAR10 Data

In [6]:
from tensorflow.keras.datasets import cifar10
(train_data, train_labels), (test_data, test_labels) = cifar10.load_data()
print(train_data.shape, train_labels.shape)
print(test_data.shape, test_labels.shape)

train_data = train_data.astype(np.double)
train_data = train_data / 255.0

test_data = test_data.astype(np.double)
test_data = test_data / 255.0

170500096/170498071 [==============================] - 2s 0us/step
(50000, 32, 32, 3) (50000, 1)
(10000, 32, 32, 3) (10000, 1)


> ### Preprocessiing

In [7]:
train_data = tf.image.resize(train_data, (IMG_SIZE, IMG_SIZE))
test_data = tf.image.resize(test_data, (IMG_SIZE, IMG_SIZE))
print(train_data.shape, test_data.shape)

(50000, 64, 64, 3) (10000, 64, 64, 3)


> ### Compile Model

In [8]:
model_fine.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

> ### 학습 (Training)

In [ ]:
%%time
history = model_fine.fit(train_data, train_labels, shuffle=True,
                batch_size=500, epochs=30,
                validation_data=(test_data, test_labels))

Epoch 1/30
100/100 [==============================] - 1570s 16s/step - loss: 0.5584 - accuracy: 0.8133 - val_loss: 1.7688 - val_accuracy: 0.5805
Epoch 2/30
100/100 [==============================] - 1545s 15s/step - loss: 0.2086 - accuracy: 0.9280 - val_loss: 1.0888 - val_accuracy: 0.7065
Epoch 3/30
100/100 [==============================] - 1551s 16s/step - loss: 0.1368 - accuracy: 0.9533 - val_loss: 0.5502 - val_accuracy: 0.8466
Epoch 4/30
 64/100 [==================>...........] - ETA: 8:53 - loss: 0.0983 - accuracy: 0.9651

> ### Ploting

In [ ]:
loss = history.history['val_loss']
epochs = range(1, len(loss)+1)

plt.figure(figsize=(10, 10))
plt.subplot(2, 1, 1)
plt.title('Accuray')
plt.plot(epochs, history.history['accuracy'], 'r', label='accuracy')
plt.plot(epochs, history.history['val_accuracy'], 'g', label='val_accuracy')
plt.grid(True)
plt.ylabel('Accuracy')
plt.legend(loc='best')

plt.subplot(2, 1, 2)
plt.title('Loss')
plt.plot(epochs, history.history['loss'], 'r', label='loss')
plt.plot(epochs, history.history['val_loss'], 'g', label='val_loss')
plt.grid(True)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='best')
plt.show()

> ### 결과 출력을 위한 함수

In [ ]:
def Make_Result_Plot(suptitle, data, label, y_max):
    fig_result, ax_result = plt.subplots(2, 5, figsize=(18, 7))
    fig_result.suptitle(suptitle)
    for idx in range(10):
        ax_result[idx//5][idx%5].imshow(data[idx],cmap="binary")
        ax_result[idx//5][idx%5].set_title("test_data[{}] (label : {} / y : {})".format(idx, label[idx], y_max[idx]))

> ### 학습 후 상황

In [ ]:
y_out = model_fine.predict(test_data)
y_max = np.argmax(y_out, axis=1).reshape((-1, 1))
Make_Result_Plot("After Training", test_data, test_labels, y_max)